In [7]:
from preprocess import TextReader
from train import CNNText

import pandas as pd
from pyemd import emd
import numpy as np
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split

## 1. Loading the pretrained word vector

In [ ]:
model = KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin',
                                         binary=True)

## 2. Process the data and convert words into vectors

In [16]:
def process_word_vectors(base_path, suffix, pretrained=False, **kwargs):
    tr = TextReader(data_dir=base_path, 
                    suffix_labels=suffix)
    print(tr.data_files)
    if tr.prepare_data(clean=True):
        X, y = tr.get_ranked_features()
    word_vectors_df = None
    if pretrained:
        model = kwargs.get('model')
        if model is None:
            raise ValueError('Model can not be None')
        wv = tr.get_embedding_vector(model)
        word_vectors = {}
        for word, vector in wv:
            word_vectors[tr.get_rank(word)] = vector
        word_vectors_df = pd.DataFrame.from_dict(word_vectors, orient='index')
    return X, y, word_vectors_df

In [17]:
!ls

CNN_4_RNN.ipynb __pycache__     model.py        ranks.npy       rt-polarity.pos
README.md       batch.py        preprocess.py   rt-polarity.neg train.py


In [18]:
X, y, _ = process_word_vectors('./', 
                                suffix={'rt-polarity.pos': 1, 
                                        'rt-polarity.neg': 0})

{'./rt-polarity.pos': 1, './rt-polarity.neg': 0}


100%|██████████| 5331/5331 [00:19<00:00, 278.71it/s]


In [19]:
X.shape

(10662, 51)

In [20]:
y.shape

(10662,)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
%%sh

mkdir train valid

In [23]:
np.save('./train/X_train', X_train)
np.save('./train/y_train', y_train)

np.save('./valid/X_valid', X_test)
np.save('./valid/y_train', y_test)

In [24]:
!ls train/

X_train.npy y_train.npy


In [25]:
!ls valid/

X_valid.npy y_train.npy


## Testing the batch Iterators

In [26]:
cnnText = CNNText(
    train_path='./train/',
    valid_path='./valid/',
    epochs=50,
    batch_size=50
)

In [27]:
cnnText.train()

Epoch 50: 100%|##########| 170/170 [00:00<00:00, 210.64it/s, train_loss=21.9, valid_loss=21.9]


In [116]:
batch_size = 3
epochs = 5

In [117]:
# N // batch_size 

In [ ]:
def train(epochs, batch_size, X, y):
    N = X.shape[0]
    pointer = 0
    for e in range(epochs):
        x_out = X[pointer: pointer+batch_size, :]
        y_out = y[pointer: pointer+batch_size]
#         for bs in range(batch_size):
        x_out.append(X[(pointer + bs) % N])
        print(x_out)
        pointer += batch_size
        

In [129]:
pointer = 0

In [131]:
y[0:3]

array([0, 1, 1])

In [130]:
.shape

(3, 51)

In [133]:
import numpy as np

In [ ]:
np.loa

In [80]:
[*range(0, 65, 3)]

[0,
 3,
 6,
 9,
 12,
 15,
 18,
 21,
 24,
 27,
 30,
 33,
 36,
 39,
 42,
 45,
 48,
 51,
 54,
 57,
 60,
 63]